In [68]:
from elasticsearch import Elasticsearch
from sentence_transformers import SentenceTransformer
from tqdm.notebook import tqdm
import json
import numpy as np
import pandas as pd

## Q1. Getting the embeddings model

In [47]:
model_name = "multi-qa-distilbert-cos-v1"
user_question = "I just discovered the course. Can I still join it?"
documents_path = "../eval/documents-with-ids.json"
ground_truth_path = "../eval/ground-truth-data.csv"

In [3]:
embedding_model = SentenceTransformer(model_name)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/9.52k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/523 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/333 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

What's the first value of the resulting vector?

In [31]:
v = embedding_model.encode(user_question)
v[0]

0.07822266

## Prepare the documents

We will use only a subset of the questions - the questions for "machine-learning-zoomcamp". After filtering, you should have only 375 documents

In [18]:
documents = [doc for doc in json.loads(open(documents_path).read()) if doc["course"] == "machine-learning-zoomcamp"]

In [19]:
len(documents)

375

## Q2. Creating the embeddings

In [20]:
documents[0]

{'text': 'Machine Learning Zoomcamp FAQ\nThe purpose of this document is to capture frequently asked technical questions.\nWe did this for our data engineering course and it worked quite well. Check this document for inspiration on how to structure your questions and answers:\nData Engineering Zoomcamp FAQ\nIn the course GitHub repository there’s a link. Here it is: https://airtable.com/shryxwLd0COOEaqXo\nwork',
 'section': 'General course-related questions',
 'question': 'How do I sign up?',
 'course': 'machine-learning-zoomcamp',
 'id': '0227b872'}

In [23]:
embeddings = [
    embedding_model.encode(f"{doc['question']} {doc['text']}")
    for doc in tqdm(documents)
]

  0%|          | 0/375 [00:00<?, ?it/s]

In [25]:
X = np.array(embeddings)

What's the shape of X? (X.shape). Include the parantheses.

In [26]:
X.shape

(375, 768)

## Q3. Search

We have the embeddings and the query vector. Now let's compute the cosine similarity between the vector from Q1 (let's call it v) and the matrix from Q2.

The vectors returned from the embedding model are already normalized (you can check it by computing a dot product of a vector with itself - it should return something very close to 1.0). This means that in order to compute the coside similarity, it's sufficient to multiply the matrix X by the vector v.

In [30]:
X[0].dot(X[0])

1.0000001

In [32]:
scores = X.dot(v)

What's the highest score in the results?

In [33]:
max(scores)

0.6506573

## Vector search

In [42]:
class VectorSearchEngine():
    def __init__(self, documents, embeddings):
        self.documents = documents
        self.embeddings = embeddings

    def search(self, v_query, num_results=10):
        scores = self.embeddings.dot(v_query)
        # idx = np.argsort(-scores)[:num_results]
        idx = np.argpartition(-scores, num_results)[:num_results]
        return [self.documents[i] for i in idx]

In [44]:
search_engine = VectorSearchEngine(documents=documents, embeddings=X)
search_engine.search(v, num_results=5)

[{'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'section': 'General course-related questions',
  'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp',
  'id': 'ee58a693'},
 {'text': 'Welcome to the course! Go to the course page (http://mlzoomcamp.com/), scroll down and start going through the course materials. Then read everything in the cohort folder for your cohort’s year.\nClick on the links and start watching the videos. Also watch office hours from previous cohorts. Go to DTC youtube channel and click on Playlists and search for {course yyyy}. ML Zoomcamp was first launched in 2021.\nOr you c

## Q4. Hit-rate for our search engine

Now use the code from the module to calculate the hitrate of VectorSearchEngine with num_results=5

In [48]:
df_ground_truth = pd.read_csv(ground_truth_path)
df_ground_truth = df_ground_truth[df_ground_truth.course == 'machine-learning-zoomcamp']
ground_truth = df_ground_truth.to_dict(orient='records')

In [49]:
ground_truth[0]

{'question': 'Where can I sign up for the course?',
 'course': 'machine-learning-zoomcamp',
 'document': '0227b872'}

In [58]:
relevance_total = [
    [
        d["id"] == record["document"] 
        for d in search_engine.search(
            v_query=embedding_model.encode(record["question"]),
            num_results=5
        )
    ]
    for record in tqdm(ground_truth)
]

  0%|          | 0/1830 [00:00<?, ?it/s]

In [66]:
def hit_rate(relevance_total):
    return sum([any(r) for r in relevance_total]) / len(relevance_total)

What did you get?

In [67]:
hit_rate(relevance_total)

0.9398907103825137

## Q5. Indexing with Elasticsearch

In [69]:
es_client = Elasticsearch('http://localhost:9200')

In [70]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
            "id": {"type": "keyword"},
            "question_text_vector": {
                "type": "dense_vector",
                "dims": 768,
                "index": True,
                "similarity": "cosine"
            },
        }
    }
}

index_name = "course-questions"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [80]:
for doc, emb in zip(tqdm(documents), embeddings):
    es_client.index(index=index_name, document={**doc, "question_text_vector": emb})

  0%|          | 0/375 [00:00<?, ?it/s]

After indexing, let's perform the search of the same query from Q1.

In [90]:
def elastic_search_knn(es_client: Elasticsearch, index_name, vector, k=5):
    search_query = {
        "knn": {
            "field": "question_text_vector",
            "query_vector": vector,
            "k": k,
            "num_candidates": 10000,
        },
        "_source": ["text", "section", "question", "course", "id"]
    }
    return es_client.search(index=index_name, body=search_query)
    

What's the ID of the document with the highest score?

In [91]:
elastic_search_knn(es_client, index_name, v)["hits"]["hits"][0]["_source"]["id"]

'ee58a693'

## Q6. Hit-rate for Elasticsearch

In [93]:
relevance_total_es = [
    [
        d["_source"]["id"] == record["document"] 
        for d in elastic_search_knn(
            es_client,
            index_name,
            vector=embedding_model.encode(record["question"]),
            k=5,
        )["hits"]["hits"]
    ]
    for record in tqdm(ground_truth)
]

  0%|          | 0/1830 [00:00<?, ?it/s]

What's hitrate for our dataset for Elastic?

In [95]:
hit_rate(relevance_total_es)

0.9398907103825137